## 이투데이 vs. 연합뉴스 기사 유사도 비교

- 연합뉴스 일정기간 기사 원문을 Scikit-learn의 TFIDF Vectorizer로 train(fit)하여 Vocabulary와 IDF를 만들고
- Dcoument Term Matrix(DTM)을 계산(transform)
- train된 DTM에 이투데이 기사를 transform하여 생성된 DTM을 train DTM과 dot product하여 cosine 유사도 계산

In [1]:
result_data_prefix = '이투데이-연합뉴스-유사도_3_1000'

### Mecab Tokenizer
- 기사 원문의 모든 단어/어절을 vector화 할 경우연산이 많아지며, 문맥에 무의미한 불용어(stopword)도 포함되는 문제를 해소하기 위해
- Mecab을 사용하여 문장 중 유의미한 품사만 token으로 사용하고, Mecab 사전에 등록된 한자단어가 NNG로 분석되는 특성을 회피하기 위해 '한자 배제' 정규식 적용
- 추가로 stopword를 지정하여 불용어 제거 (현재는 Mecab의 분석 결과가 특별한 불용어 처리가 필요하지 않아 비워두었음)
- 비교 테스트를 위해 KoNLPy의 Mecab을 사용하였으나, 단독 버전 MeCab으로 사용하는 것이 메모리 소요나 호출 시간에 유리하지 않을지 고민 중...

### 정환 메모

바로 아래 코드는 회사에서 만든 토크나이저 파일이 필요하다. 받은 파일이 없다면 그냥 넘어갈 것!

In [ ]:
import os
import sys
#sys.path.append('D:/Projects/WIGO/Tokenization')
# sys.path.append(os.path.dirname(os.path.abspath('../similarity/tokenization')))
from tokenization.tokenizer import Tokenizer

tokenizer = Tokenizer(
    tagger="Mecab",
    pos=["NNG","NNP", "NP", "XPN", "XR", "VV", "VA","VX", "VCP","VCN", "XSV", "XSA", "MM", "MAG", "MAJ", "IC", "SN"]
)
' '.join(tokenizer.tokenize('우리는 민족중흥의 역사적 사명을 띠고 이 땅에 태어났다. 금일 코로나 환자 19명 발생!!!'))

### Multi-Processing

### Train Data 로드

- 현재 시나리오에 맞추어 file_name, title, body 로 고정된 CSV인데
- 일반화를 위해 CSV 구조를 확장 가능하도록 설계하거나
- CSV 로드 부분을 메인로직에서 빼내어 task별로 별도의 모듈로 만드는 방안 수립 필요

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import joblib
import os

np.random.seed(0)

# train_data_path = 'D:/Projects/_Corpus/DataBoucher_20201110/yeonhap.csv'
train_data_path = './pretreatment_data/yeonhap_pre.csv'
# 학습용 데이터를 로드하고 DataFrame 생성
train_df = pd.read_csv(train_data_path, encoding='utf-8', names=['CNo', 'Subject', 'Contents'])
#train_df.head()
#print(train_df.info())
test = [df.split(' ') for df in train_df['Contents']]

print(test[0:5])

[['합동', '차례', '민속놀이', '영상', '통화', '향수', '주민', '초청', '행사', '생략', '이역만리', '파병', '장병', '코로나', '와중', '추석', '합동', '차례', '민속놀이', '영상', '통화', '향수', '주민', '초청', '행사', '생략', '유현민', '기자', '이역만리', '구슬땀', '천', '해외', '파병', '장병', '민족', '최대', '명절', '한가위', '합동', '참모', '본부', '군', '운용', '해외', '파병', '부대', '장병', '추석', '합동', '차례', '가족', '영상', '통화', '고국', '그리움', '올해', '신종', '코로나', '바이러스', '감염증', '예방', '예년', '외부', '활동', '현지', '주민', '초청', '행사', '레바논', '유엔', '평화', '유지', '활동', '동명', '부대', '감시', '정찰', '임무', '평소', '수행', '합동', '차례', '대형', '윷놀이', '투호', '민속놀이', '가족', '아쉬움', '아프리카', '남수단', '재건', '지원', '활동', '한빛', '부대', '합동', '차례', '윷놀이', '제기차기', '명절', '분위기', '소말리아', '아덴만', '해역', '다국적군', '상선', '보호', '임무', '수행', '청해부대', '군수품', '보급', '오만', '기항', '추석', '부대', '다음', '항해', '준비', '합동', '차례', '민속놀이', '영상', '통화', '프로그램', '마련', '아랍에미리트', '에', '파병', '아크', '부대', '장병', '최근', '군', '연합', '훈련', '성공', '고국', '가족', '영상', '통화', '추석', '아크', '부대장', '박용규', '중령', '코로나', '제한', '상황', '부여', '임무', '성공', '완수', '장병', '사기', '가족', '해외', '우리', '국군', 

### TfidfVectorizer 생성

- 한글의 경우 lowercase는 False로 설정하는 것이 좋음 (다른 Task에서의 경험, 일부 깨지는 현상이 있을 수 있음)
- ngram은 1,2 등 다른 값을 주어 테스트 해 보았으나, 형태소 분석을 거친 token의 경우 1,1이 가장 성능(실행시간, 유사도 모두)이 좋음
- max_feature는 50,000으로 지정하였으나, train set 저장을 고려할 때 Database 전체를 하나의 train set으로 만들 수 있는지 실험 필요(아마도 어려울 것)
- preprocessor는 좋은 개념으로 보여 일단 선언만 해 두었으며, 이후 추가 study 필요
- vocabulary는 미리 생성하여 지정하면, fit 시간이 줄어드는지 여부 확인 필요

### 정환 메모

TfidfVectorizer 옵션을 설정하는 코드

따로 토크나이저 파일을 받지 않았다면 tokenizer=, token_patten=, vocabulary= 이 3가지 옵션은 주석처리 하자!

궁금증 => 외부 토크나이저는 어떻게 사용할까?

In [3]:
tfidf_vectorizer = TfidfVectorizer(
    lowercase=False, # 소문자로 바꿔줌, 한글에서는 사용 X (default = True)
    preprocessor=None, # 
    # tokenizer=tokenizer,
    # tokenizer=[df.split(' ') for df in train_df['Contents']],
    analyzer = 'word', # 학습단위를 'word' 또는 'char'로 지정
    # stop_words=tokenizer.stopword,
    # token_pattern=None, # tokenizer가 선언되면 token_pattern은 None처리할 것
    ngram_range = (1, 1), # 1단어부터 n단어까지 묶음으로 Vocabulary 생성
    #max_df = 0.80 #: 문서의 80% 이상에 나타나는 단어 무시
    # max_df = 10 : 10개 이상의 문서에 나타나는 단어 무시
    max_df=1000,
    #min_df = 0.01 #: 문서의 1% 미만으로 나타나는 단어 무시
    # min_df = 10 : 문서 10개 미만에 나타나는 단어 무시
    min_df=3, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거
    max_features = 50000, # Features의 갯수를 제한
    # vocabulary=None,
    smooth_idf=True,
    sublinear_tf=True, # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)


### Training

- 실험 조건 51,864개 기사 69.8MB 학습 시간이 fit 30-36초 transform 32-38초 소요
- 학습 후 Pickle 또는 Joblib로 저장하는 방안 고려 필요
- 저장을 고려할 때 fit 한 상태로 저장할 것인지, transform 한 상태로 저장할 것인지 확인 필요

In [4]:
# training set으로부터 vocabulary와 idf를 학습하고 Document Term Matrix(이하 DTM)을 리턴
tfidf_matrix = tfidf_vectorizer.fit_transform(train_df['Contents'])

print(tfidf_matrix.shape)

(64122, 34548)


### Dump to File

- joblib를 이용하여 압축률 3으로 Prdiction에 필요한 객체(train_df, tfidf_matrix, tfidf_vectorizer)를 저장

### 정환 메모

joblib와 pickle 뭐가 다를까?

joblib 가 최신 버전! joblib도 내부를 보면 pickle를 사용함.

상황에 따라 사용하자

In [5]:
# compress: 0~9의 int 또는 bool (True이면 3) 또는 2-Tuple
# protocol: 버전 3.0에서 기본 프로토콜은 3 (버전 3.8에서 기본 프로토콜은 4)
joblib.dump((train_df, tfidf_matrix, tfidf_vectorizer), f'{result_data_prefix}-train.joblib', compress=True, protocol=3)

# Memory 해제
del train_df
del tfidf_matrix
del tfidf_vectorizer

### Load from File

- Joblib로 저장된 객체를 읽고 Instance화

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import numpy as np
import joblib
import csv
import math
import multiprocessing
from multiprocessing import Pool, Queue
from collections import defaultdict
from time import sleep
from tqdm.notebook import tqdm, trange

np.random.seed(0)

alloted_CPUs = multiprocessing.cpu_count() * 0.5
num_cores = int(alloted_CPUs + alloted_CPUs % 2)

train_df, tfidf_matrix, tfidf_vectorizer = joblib.load(f'{result_data_prefix}-train.joblib')

### Similarity 연산 및 결과 저장

- 현재 시나리오에서는 학습과 적용, 저장이 분리될 필요가 없어 운용에 편의를 위해 하나의 파일에 구현하였으나
- 일반화를 고려할 때 별도의 API에 구현되어야 할 내용

In [7]:
def CalcSimilarity(tfidf_matrix, train_df, tfidf_vectorizer, q):
    target_matrix = tfidf_vectorizer.transform([target[2].replace('ㆍ','*')])

    # dot product 후 crs_matrix를 nparray로 변환하고 차원을 축소 1차원 array로 변환
    scores = (tfidf_matrix * target_matrix.T).toarray().sum(axis=1)

    for j in scores.argsort()[::-1]:
        if scores[j] >= interval_min and scores[j] < interval_max: # min 이상 max 미만
            itemDict = {'targetFilename':'', 'trainFilename':'', 'score':''}
            itemDict['targetFilename'] = target[0]
            itemDict['trainFilename'] = train_df._get_value(j, 'file_name')
            itemDict['score'] = math.floor(scores[j] * 1000) / 1000
            q.put(itemDict)
    q.put('STOP')

#### Target Dataframe으로 for loop을 돌며 cosine similarity 연산하고 row 별로 직접 저장

In [ ]:
print(target_df)

In [8]:
# target_data_path = 'D:/Projects/_Corpus/DataBoucher_20201110/etoday.csv'
target_data_path = './pretreatment_data\etoday_pre.csv'

# Target도 같은 tfidf_vectorizer로 transform하고 dot product
target_df = pd.read_csv(target_data_path, encoding='utf-8', names=['CNo', 'Subject', 'Contents'])

# 유사도 산출 구간을 min =< ~ > max로 지정
# 실험결과 1.0인 경우 Python의 특성상 소수점 아래 가비지가 발생하여 2.0으로 비교할 것
interval_min = 0.7
interval_max = 0.8

# Data의 양이 많으므로 DataFrame을 만들지 않고 직접 저장
with open(f'{result_data_prefix}-{interval_min}_{interval_max}.csv', 'wt', encoding='utf-8', newline='') as file_csv:
    writer = csv.writer(file_csv, delimiter=',', lineterminator='\n') # 콤마(,)를 delimiter로 사용
    for i in tqdm(target_df.index, desc='target_df loop: ', position=0):
        # train vectorizer에 target을 transform
        target_matrix = tfidf_vectorizer.transform([target_df._get_value(i, 'Contents')]).astype(np.float16)

        # dot product 후 crs_matrix를 nparray로 변환하고 차원을 축소 1차원 array로 변환 (cosine_similarity 보다 다소 빠르고 score는 다소 작게 계산됨)
        scores = (tfidf_matrix * target_matrix.T).toarray().reshape(-1,)
        # cosine_similarity 연산 후 nparray 차원을 축소 1차원 array로 변환
        # scores = cosine_similarity(tfidf_matrix, target_matrix).reshape(-1,)

        for j in scores.argsort()[::-1]: # 내림차순 정렬
            score = math.floor(scores[j] * 10000) / 10000 # 소수점 두자리 이하 버림
            if score >= interval_min and \
                ((interval_max < 1.0 and score < interval_max) or (interval_max == 1.0 and score <= interval_max)): # min 이상 max 미만, max가 1일 경우는 max 이하
                #print('{} and {} / score : {}'.format(target_df._get_value(i, 'CNo'), train_df._get_value(j, 'CNo'), score))
                writer.writerow([target_df._get_value(i, 'CNo'), train_df._get_value(j, 'CNo'), score]) # 하나의 row를 리스트 형태로 입력
            elif interval_min > score:
                break

# Memory 해제
#del target_df
#del train_df
#del tfidf_matrix
#del tfidf_vectorizer

target_df loop:   0%|          | 0/9747 [00:00<?, ?it/s]

In [ ]:
result = pd.read_csv("")

아래는 소장님이 개인적으로 이것저것 실험해 본 코드라서 볼 필욘 없다

In [ ]:
import pandas as pd
import numpy as np
import time

#del df1, df2, df3, df4
numOfRows = 1000
"""
# append
startTime = time.perf_counter()
df1 = pd.DataFrame(np.random.randint(100, size=(5,5)), columns=['A', 'B', 'C', 'D', 'E'])
for i in range( 1,numOfRows-4):
    df1 = df1.append( dict( (a,np.random.randint(100)) for a in ['A','B','C','D','E']), ignore_index=True)
print('Elapsed time: {:6.3f} seconds for {:d} rows'.format(time.perf_counter() - startTime, numOfRows))
print(df1.shape)

# .loc w/o prealloc
startTime = time.perf_counter()
df2 = pd.DataFrame(np.random.randint(100, size=(5,5)), columns=['A', 'B', 'C', 'D', 'E'])
for i in range( 1,numOfRows):
    df2.loc[i]  = np.random.randint(100, size=(1,5))[0]
print('Elapsed time: {:6.3f} seconds for {:d} rows'.format(time.perf_counter() - startTime, numOfRows))
print(df2.shape)

# .loc with prealloc
df3 = pd.DataFrame(index=np.arange(0, numOfRows), columns=['A', 'B', 'C', 'D', 'E'] )
startTime = time.perf_counter()
for i in range( 1,numOfRows):
    df3.loc[i]  = np.random.randint(100, size=(1,5))[0]
print('Elapsed time: {:6.3f} seconds for {:d} rows'.format(time.perf_counter() - startTime, numOfRows))
print(df3.shape)
"""
# dict
startTime = time.perf_counter()
row_list = []
#for i in range (0,5):
#    row_list.append(dict((a,np.random.randint(100)) for a in ['A','B','C','D','E']))

for i in range(0, numOfRows):
    dict1 = dict((a,np.random.randint(100)) for a in ['A','B','C','D','E'])
    row_list.append(dict1)

df4 = pd.DataFrame(row_list, columns=['A','B','C','D','E'])
print('Elapsed time: {:6.3f} seconds for {:d} rows'.format(time.perf_counter() - startTime, numOfRows))
print(df4.shape)
#print(row_list)
print(df4)

In [ ]:
import random
import pandas as pd
import numpy as np
from multiprocessing import  Pool

In [ ]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, 4)
    pool = Pool(4)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def add_features(df):
    df['question_text'] = df['question_text'].apply(lambda x:str(x))
    df["lower_question_text"] = df["question_text"].apply(lambda x: x.lower())
    df['total_length'] = df['question_text'].apply(len)
    df['capitals'] = df['question_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['total_length']),
                                axis=1)
    df['num_words'] = df.question_text.str.count('\S+')
    df['num_unique_words'] = df['question_text'].apply(lambda comment: len(set(w for w in comment.split())))
    df['words_vs_unique'] = df['num_unique_words'] / df['num_words'] 
    df['num_exclamation_marks'] = df['question_text'].apply(lambda comment: comment.count('!'))
    df['num_question_marks'] = df['question_text'].apply(lambda comment: comment.count('?'))
    df['num_punctuation'] = df['question_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
    df['num_symbols'] = df['question_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
    df['num_smilies'] = df['question_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
    df['num_sad'] = df['question_text'].apply(lambda comment: sum(comment.count(w) for w in (':-<', ':()', ';-()', ';(')))
    df["mean_word_len"] = df["question_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
train_df = pd.read_csv("D:/Projects/_Corpus/DataBoucher/etoday.csv")

In [ ]:
%%timeit
train = parallelize_dataframe(train_df, add_features) 

In [ ]:
%%timeit
train = add_features(train_df)

In [ ]:
SELECT_ARTICLE_PROCEDURE = '''
    EXEC EYE_COMMON.dbo.ArticleSimilarity_sel @startdate = '%s', @enddate = '%s', @sct = '%s'
'''

print(SELECT_ARTICLE_PROCEDURE % ('2020-01-01', '2020-12-31', 'AA001'))